Lets use MAMS dataset

In [31]:
import warnings
import sys
sys.path.append("/Users/hd/Desktop/EMOTION-PRED/src")

from emotion import run_full_emotion_pipeline
warnings.filterwarnings("ignore")

In [32]:
from datasets import load_dataset

# load the dataset
ds = load_dataset("NEUDM/mams")

# print dataset info
print(ds)

Repo card metadata block was not found. Setting CardData to empty.


DatasetDict({
    train: Dataset({
        features: ['task_type', 'dataset', 'input', 'output', 'situation', 'label', 'extra', 'instruction'],
        num_rows: 7446
    })
    validation: Dataset({
        features: ['task_type', 'dataset', 'input', 'output', 'situation', 'label', 'extra', 'instruction'],
        num_rows: 900
    })
    test: Dataset({
        features: ['task_type', 'dataset', 'input', 'output', 'situation', 'label', 'extra', 'instruction'],
        num_rows: 900
    })
})


In [33]:
train_ds = ds["train"]
train_ds = train_ds.to_pandas()


# inspect row and column count
print(f"Number of rows: {train_ds.shape[0]}")
print(f"Number of columns: {train_ds.shape[1]}")

# inspect column names
print("Column names:", train_ds.columns.tolist())

Number of rows: 7446
Number of columns: 8
Column names: ['task_type', 'dataset', 'input', 'output', 'situation', 'label', 'extra', 'instruction']


In [34]:
# Columns to keep
keep_cols = ["input", "output"]
clean_df = train_ds[keep_cols].copy()


In [35]:
import ast
import pandas as pd

# Take 50 rows
df_head = clean_df["input"].head(50).to_frame()

# 1) Convert "['text']" → "text"
def clean_list_string(x):
    try:
        parsed = ast.literal_eval(x)   # safely parse list-string
        if isinstance(parsed, list) and len(parsed) > 0:
            return parsed[0]
        return x
    except:
        return x

df_head["sentence"] = df_head["input"].apply(clean_list_string)

# 2) Add empty aspect column (required by pipeline)
df_head["aspect_term"] = ""

# 3) Save in the correct 2-column format
input_csv = "/Users/hd/Desktop/EMOTION-PRED/src/data/temp_50.csv"
df_head[["sentence", "aspect_term"]].to_csv(input_csv, index=False)

df_head.head()

,input,sentence,aspect_term
0,[It might be the best sit down food I've had i...,[It might be the best sit down food I've had i...,
1,[Hostess was extremely accommodating when we a...,[Hostess was extremely accommodating when we a...,
2,[We were a couple of minutes late for our rese...,[We were a couple of minutes late for our rese...,
3,"[Though the service might be a little slow, th...","[Though the service might be a little slow, th...",
4,[Although we arrived at the restaurant 10 min ...,[Although we arrived at the restaurant 10 min ...,


In [36]:
from emotion import run_full_emotion_pipeline

run_full_emotion_pipeline(
    input_csv=input_csv,
    dataset_name="sample50",
    results_root="/Users/hd/Desktop/EMOTION-PRED/src/results/"
)


🚀 Starting full emotion pipeline

📁 Saving outputs to: /Users/hd/Desktop/EMOTION-PRED/src/results/emotion_sample50

🔹 Annotating with: j-hartmann/emotion-english-distilroberta-base
  🔍 Model type: roberta | arch=['RobertaForSequenceClassification']
   ✅ Saved → /Users/hd/Desktop/EMOTION-PRED/src/results/emotion_sample50/j_hartmann_emotion_english_distilroberta_base_annotated.csv

🔹 Annotating with: j-hartmann/emotion-english-roberta-large
  🔍 Model type: roberta | arch=['RobertaForSequenceClassification']
   ✅ Saved → /Users/hd/Desktop/EMOTION-PRED/src/results/emotion_sample50/j_hartmann_emotion_english_roberta_large_annotated.csv

🔹 Annotating with: nateraw/bert-base-uncased-emotion
  🔍 Model type: bert | arch=['BertForSequenceClassification']
   ✅ Saved → /Users/hd/Desktop/EMOTION-PRED/src/results/emotion_sample50/nateraw_bert_base_uncased_emotion_annotated.csv

🔹 Annotating with: joeddav/distilbert-base-uncased-go-emotions-student
  🔍 Model type: distilbert | arch=['DistilBertForSe

OSError: mrm8488/t5-large-finetuned-emotion is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
import os
import pandas as pd

RESULT_DIR = "/Users/hd/Desktop/EMOTION-PRED/src/results/emotion_sample50"

csv_files = [f for f in os.listdir(RESULT_DIR) if f.endswith(".csv")]

MODEL_NAME_MAP = {
    "j_hartmann_emotion_english_roberta_large": "roberta_large",
    "nateraw_bert_base_uncased_emotion": "bert_base_emotion",
    "j_hartmann_emotion_english_distilroberta_base": "roberta_emotion",
    "joeddav_distilbert_base_uncased_go_emotions_student": "go_emotions",
    "cardiffnlp_twitter_roberta_base_emotion": "tweet_eval"
}

merged = pd.DataFrame()

for f in csv_files:
    full_path = os.path.join(RESULT_DIR, f)
    df = pd.read_csv(full_path)

    base = f.replace("_annotated.csv", "")
    colname = MODEL_NAME_MAP.get(base, base)   # fallback = raw name if missing

    merged[colname] = df["emotion_auto"]

pd.set_option("display.max_colwidth", None)
print(merged.head(20))